# Load Packages



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 

# Load DataFrame

In [ ]:
train = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv", dtype=str)
print(train.shape)

In [ ]:
train.head(10)

# Label Distribution

In [ ]:
y_train = train.label

(train.label.value_counts() / len(train)).to_frame().T

# View Sample of Images

In [ ]:
# Sample 16 images from the training set and display these along with their labels.

plt.figure(figsize=(10,10)) # specifying the overall grid size

for i in range(16):
    plt.subplot(4,4,i+1)    # the number of images in the grid is 6*6 (16)
    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][i]}.tif')
    plt.imshow(img)
    plt.text(0, -5, f'Label {train["label"][i]}')
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Split & Sample Data

In [ ]:
train_neg = train[train['label']=='0'].sample(4000,random_state=1)
train_pos = train[train['label']=='1'].sample(4000,random_state=1)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train = shuffle(train_data)

In [ ]:
train['label'].value_counts()

In [ ]:
# function to apply the .tif extension
def append_ext(fn):
    return fn+".tif"


train['id'] = train['id'].apply(append_ext)
train.head()

In [ ]:
# Split the dataframe train into two DataFrames named train_df and valid_df. 
# Use 20% of the data for the validation set. 
# Use stratified sampling so that the label proportions are preserved.
# Set a random seed for the split. 

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=1, stratify=train.label)

print(train_df.shape)
print(valid_df.shape)

In [ ]:
# Create image data generators for both the training set and the validation set. 
# Use the data generators to scale the pixel values by a factor of 1/255. 
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
# Complete the code for the data loaders below. 

BATCH_SIZE = 64

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = '../input/histopathologic-cancer-detection/train/',
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = train_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = '../input/histopathologic-cancer-detection/train/',
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

In [ ]:
# Run this cell to determine the number of training and validation batches. 

TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

# Build Network

In [ ]:
# Use this cell to construct a convolutional neural network model. 
# Your model should make use of each of the following layer types:
#    Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense
# You can start by mimicking the architecture used in the 
# Aerial Cactus competetition, but you should explore different architectures
# by adding more layers and/or adding more nodes in individual layers

np.random.seed(1)
tf.random.set_seed(1)

cnn1 = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.25),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    # we have 2 here because we have 2 classes
    Dense(2, activation='softmax')
])

cnn1.summary()

# Train Network

In [ ]:
opt = tf.keras.optimizers.Adam(0.001)
cnn1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

# Training Run 1

In [ ]:
%%time 

h1 = cnn1.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
history = h1.history
print(history.keys())

In [ ]:
# Graph the result

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])

plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()

plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()

plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()

plt.tight_layout()
plt.show()

# Model 2

### Build Network

In [ ]:
# Use this cell to construct a convolutional neural network model. 
# Your model should make use of each of the following layer types:
#    Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense
# You can start by mimicking the architecture used in the 
# Aerial Cactus competetition, but you should explore different architectures
# by adding more layers and/or adding more nodes in individual layers

np.random.seed(1)
tf.random.set_seed(1)

cnn2 = Sequential([
    Conv2D(32, (3,3), activation = 'relu', padding = 'same', input_shape=(32,32,3)),
    Conv2D(32, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.20),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.6),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.6),
    BatchNormalization(),

    Flatten(),
    
    Dense(32, activation='relu'),
    Dropout(0.6),
    Dense(16, activation='relu'),
    Dropout(0.20),
    BatchNormalization(),
    # we have 2 here because we have 2 classes
    Dense(2, activation='softmax')
])

cnn2.summary()

### Train Network

In [ ]:
opt = tf.keras.optimizers.Adam(0.001)
cnn2.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
%%time 

h2 = cnn2.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 50,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
history = h2.history
print(history.keys())

In [ ]:
# Graph the result

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])

plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()

plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()

plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc_1'], label='Training')
plt.plot(epoch_range, history['val_auc_1'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()

plt.tight_layout()
plt.show()

# Save Model and History

In [ ]:

cnn2.save('cancer_model.h5')
pickle.dump(history, open(f'cancer_history.pkl', 'wb'))